In [51]:
!pip install datasets

In [52]:
import pandas as pd
from datasets import load_dataset

In [53]:
sev_data= load_dataset("NEUDM/semeval-2014")
mams_data= load_dataset("NEUDM/mams")

Repo card metadata block was not found. Setting CardData to empty.


In [54]:
sev_data

DatasetDict({
    train: Dataset({
        features: ['task_type', 'dataset', 'input', 'output', 'situation', 'label', 'extra', 'instruction'],
        num_rows: 2503
    })
    validation: Dataset({
        features: ['task_type', 'dataset', 'input', 'output', 'situation', 'label', 'extra', 'instruction'],
        num_rows: 727
    })
    test: Dataset({
        features: ['task_type', 'dataset', 'input', 'output', 'situation', 'label', 'extra', 'instruction'],
        num_rows: 279
    })
})

In [55]:
df_sev_data_train = pd.DataFrame(sev_data['train'])
df__sev_data_validation = pd.DataFrame(sev_data['validation'])
df_sev_data_test = pd.DataFrame(sev_data['test'])

# Concatenate all splits
df_sev2014_full = pd.concat([df_sev_data_train, df__sev_data_validation, df_sev_data_test])

In [56]:
sem_eval_data= df_sev2014_full[['input','output']]

In [57]:
mams_data

DatasetDict({
    train: Dataset({
        features: ['task_type', 'dataset', 'input', 'output', 'situation', 'label', 'extra', 'instruction'],
        num_rows: 7446
    })
    validation: Dataset({
        features: ['task_type', 'dataset', 'input', 'output', 'situation', 'label', 'extra', 'instruction'],
        num_rows: 900
    })
    test: Dataset({
        features: ['task_type', 'dataset', 'input', 'output', 'situation', 'label', 'extra', 'instruction'],
        num_rows: 900
    })
})

In [58]:
mams_train = pd.DataFrame(mams_data['train'])
mams_validation = pd.DataFrame(mams_data['validation'])
mams_test = pd.DataFrame(mams_data['test'])

# Concatenate all splits
mams_full = pd.concat([mams_train, mams_validation, mams_test])

In [59]:
mams_data_focus= mams_full[['input','output']]

In [60]:
print("MAMS Data Focus - Output Sample:")
print(mams_data_focus['output'].head())

print("Sem Eval Data - Output Sample:")
print(sem_eval_data['output'].head())


MAMS Data Focus - Output Sample:
0         [['food', 'positive'], ['place', 'neutral']]
1    [['staff', 'positive'], ['miscellaneous', 'neu...
2    [['miscellaneous', 'neutral'], ['staff', 'nega...
3     [['service', 'negative'], ['staff', 'positive']]
4    [['staff', 'negative'], ['miscellaneous', 'neu...
Name: output, dtype: object
Sem Eval Data - Output Sample:
0                            [['mousepad', 'neutral']]
1                              [['price', 'positive']]
2    [['screen', 'negative'], ['clicking buttons', ...
3                           [['software', 'negative']]
4                   [['extended warranty', 'neutral']]
Name: output, dtype: object


In [61]:
print("Empty Outputs in MAMS Data Focus:")
print(len(mams_data_focus[mams_data_focus['output'].apply(lambda x: x == [])]))

print("Empty Outputs in Sem Eval Data:")
print(len(sem_eval_data[sem_eval_data['output'].apply(lambda x: x == [])]))


Empty Outputs in MAMS Data Focus:
0
Empty Outputs in Sem Eval Data:
0


In [62]:
complete_dataset = pd.concat([mams_data_focus, sem_eval_data], ignore_index=True)

# Recheck empty outputs
empty_outputs_post_merge = complete_dataset[complete_dataset['output'].apply(lambda x: x == [])]
print(f"Number of rows with empty lists in 'output' after merge: {len(empty_outputs_post_merge)}")

Number of rows with empty lists in 'output' after merge: 0


In [63]:
complete_dataset.head()

,input,output
0,[It might be the best sit down food I've had i...,"[['food', 'positive'], ['place', 'neutral']]"
1,[Hostess was extremely accommodating when we a...,"[['staff', 'positive'], ['miscellaneous', 'neu..."
2,[We were a couple of minutes late for our rese...,"[['miscellaneous', 'neutral'], ['staff', 'nega..."
3,"[Though the service might be a little slow, th...","[['service', 'negative'], ['staff', 'positive']]"
4,[Although we arrived at the restaurant 10 min ...,"[['staff', 'negative'], ['miscellaneous', 'neu..."


In [96]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
/content/drive/My Drive/Combined_Data.csv

In [99]:
complete_dataset.to_csv('/content/drive/My Drive/Combined_Data.csv', index=False)

In [65]:
complete_dataset.iloc[4340]

input                         [I liked the beer selection!]
output    [['beer selection', 'positive'], ['beer s', 'n...
Name: 4340, dtype: object

In [66]:
import pandas as pd
import ast
import re

# Load your data into a pandas DataFrame
# Assuming 'data.csv' is your file, replace it with the correct file path or use your existing DataFrame
# df = pd.read_csv('data.csv')

# If you already have a DataFrame loaded, use it directly
# df = your_existing_dataframe

# Define a regex pattern to match aspect and polarity pairs
# This pattern matches both quoted and unquoted strings
pair_regex = r"\[\s*['\"]?([^'\"]+)['\"]?\s*,\s*['\"]?([^'\"]+)['\"]?\s*\]"

# Initialize a set to keep track of unique polarities
unique_polarities = set()

# Function to process extracted items using regex
def process_pairs_regex(text):
    matches = re.findall(pair_regex, text)
    for aspect, polarity in matches:
        unique_polarities.add(polarity)

# Function to safely evaluate and extract pairs from well-formed strings
def process_pairs_eval(text):
    try:
        # Safely evaluate the string to a Python object
        data_list = ast.literal_eval(text)
        for item in data_list:
            if len(item) == 2:  # Check if item is a pair
                _, polarity = item
                unique_polarities.add(polarity)
    except Exception as e:
        # If there is an error with eval, fall back to regex
        process_pairs_regex(text)

# Iterate over the 'output' column in the DataFrame
for text in complete_dataset['output']:
    process_pairs_eval(text)

# Output the results
print("Number of unique polarities:", len(unique_polarities))
print("Unique polarities:", unique_polarities)

Number of unique polarities: 4
Unique polarities: {'conflict', 'negative', 'neutral', 'positive'}


In [67]:
import pandas as pd
import ast
import re

# Load your data into a pandas DataFrame
# df = pd.read_csv('data.csv') # Replace with your actual data source

# Define a regex pattern to match aspect and polarity pairs
pair_regex = r"\[\s*['\"]?([^'\"]+)['\"]?\s*,\s*['\"]?([^'\"]+)['\"]?\s*\]"

# Initialize a set to keep track of unique polarities
unique_polarities = set()

# Function to process a list of pairs
def process_list(data_list):
    for item in data_list:
        if isinstance(item, list) and len(item) == 2:
            _, polarity = item
            unique_polarities.add(polarity)

# Function to process dictionaries
def process_dict(data_dict):
    for key in data_dict:
        value = data_dict[key]
        if isinstance(value, list):
            process_list(value)

# Function to process extracted items using regex
def process_pairs_regex(text):
    matches = re.findall(pair_regex, text)
    for aspect, polarity in matches:
        unique_polarities.add(polarity)

# Function to evaluate and extract pairs from strings
def process_pairs_eval(text):
    try:
        # Safely evaluate the string to a Python object
        data_obj = ast.literal_eval(text)
        if isinstance(data_obj, list):
            process_list(data_obj)
        elif isinstance(data_obj, dict):
            process_dict(data_obj)
    except Exception as e:
        # If there is an error with eval, fall back to regex
        process_pairs_regex(text)

# Iterate over the 'output' column in the DataFrame
for text in complete_dataset['output']:
    process_pairs_eval(text)

# Output the results
print("Number of unique polarities:", len(unique_polarities))
print("Unique polarities:", unique_polarities)

Number of unique polarities: 4
Unique polarities: {'conflict', 'negative', 'neutral', 'positive'}


In [68]:
import pandas as pd
import ast
import re

# Assuming df is your original DataFrame
# Make a copy of the DataFrame to preserve the original data
df_normalized = complete_dataset.copy()

# Define a regex pattern to match aspect and polarity pairs
pair_regex = r"\[\s*['\"]?([^'\"]+)['\"]?\s*,\s*['\"]?([^'\"]+)['\"]?\s*\]"

# Function to normalize and extract aspect-polarity pairs
def normalize_data(entry):
    normalized_pairs = []

    if isinstance(entry, str):
        try:
            # Try to safely convert string to a Python object (list or dict)
            entry = ast.literal_eval(entry)
        except (ValueError, SyntaxError):
            # If it fails, use regex to extract the pairs from the string
            matches = re.findall(pair_regex, entry)
            for match in matches:
                normalized_pairs.append(list(match))
            return normalized_pairs

    if isinstance(entry, list):
        # If the entry is a list, process each item as a pair
        for item in entry:
            if isinstance(item, list) and len(item) == 2:
                normalized_pairs.append(item)

    elif isinstance(entry, dict):
        # If the entry is a dictionary, process each value which should be a list of pairs
        for key, value in entry.items():
            if isinstance(value, list):
                for item in value:
                    if isinstance(item, list) and len(item) == 2:
                        normalized_pairs.append(item)

    return normalized_pairs

# Apply the normalization to each row in the 'output' column
df_normalized['output_normalized'] = complete_dataset['output'].apply(normalize_data)

# Now df_normalized contains a new column 'output_normalized' with the data normalized

In [69]:
df_normalized.shape

(12755, 3)

In [70]:
df_normalized.iloc[12754]

input                [Their sake list was extensive, but we were lo...
output               {'aspect_term': [['sake list', 'positive'], ['...
output_normalized    [[sake list, positive], [Purple Haze, conflict...
Name: 12754, dtype: object

In [71]:
df_normalized = df_normalized.drop(columns=["output"])

In [72]:
df_normalized.head()

,input,output_normalized
0,[It might be the best sit down food I've had i...,"[[food, positive], [place, neutral]]"
1,[Hostess was extremely accommodating when we a...,"[[staff, positive], [miscellaneous, neutral]]"
2,[We were a couple of minutes late for our rese...,"[[miscellaneous, neutral], [staff, negative]]"
3,"[Though the service might be a little slow, th...","[[service, negative], [staff, positive]]"
4,[Although we arrived at the restaurant 10 min ...,"[[staff, negative], [miscellaneous, neutral]]"


In [73]:
unique_polarities = set()

# Iterate over each row and extract the aspects
for row in df_normalized['output_normalized']:
    for item in row:
        unique_polarities.add(item[1])

print(unique_polarities)

{'conflict', 'negative', 'neutral', 'positive'}


In [74]:
unique_aspects = set()

# Iterate over each row and extract the aspects
for row in df_normalized['output_normalized']:
    for item in row:
        unique_aspects.add(item[0])

print(unique_aspects)

{'roast pork', 'desktop icons', 'spaghetti with Scallops and Shrimp', 'fires', 'secret hickory sauce', 'arm velcro', 'cream', 'mix of chili sauce and creme fraiche', 'tapas-like seafood treats', 'techie', 'Lentils', 'standard os cd', 'grilled calimari', 'Sea Eel', 'pancakes', 'base installation', 'reservation', 'ricotta cheese', 'delivery time', 'hearty lunches', 'band', 'grilled chicken', 'omelets', 'salad dressing', 'steak frites', 'Bartender', 'beet ravioli appetizer', 'Raw Bar Special', 'assortment of smoothies', 'pan', 'line', 'beans on toast', 'gourmet food', 'Spicy Tuna hand rolls', 'dinin room', 'shows', 'norm', 'braised', 'roti', 'parallels type program', 'dim sum servings', 'platters', 'eggplant parmesan', 'cup', 'duck burrito with mole sauce', 'chili guac', 'blue tiles', 'fish fillet in spicy source', 'Pizzas', 'dosa batter', 'commercial ice cream', 'drumsticks over rice', 'Lamb', 'real Olive Oil', 'tuna, lemon juice', 'salad with a delicious dressing', 'transporting', 'Outd

In [75]:
import spacy
from spacy.tokens import DocBin

In [76]:
df_normalized.input.iloc[4352]

['No table space and one of the angry neighbors decided to take matters into his own hands by throwing a bucket of water out his window and onto the patrons waiting for their tables.']

In [77]:
df_normalized.output_normalized.iloc[4352]

[['table space', 'negative'],
 ['water', 'neutral'],
 ['patrons waiting', 'neutral'],
 ['table s', 'neutral']]

In [78]:
print(df_normalized['output_normalized'].apply(lambda x: all(item[0] is None for item in x)).value_counts())
print(df_normalized['output_normalized'].head(10))  # Display the first 10 rows to inspect

output_normalized
False    12755
Name: count, dtype: int64
0                 [[food, positive], [place, neutral]]
1        [[staff, positive], [miscellaneous, neutral]]
2        [[miscellaneous, neutral], [staff, negative]]
3             [[service, negative], [staff, positive]]
4        [[staff, negative], [miscellaneous, neutral]]
5         [[miscellaneous, negative], [food, neutral]]
6    [[food, neutral], [price, positive], [miscella...
7         [[miscellaneous, negative], [food, neutral]]
8        [[miscellaneous, neutral], [staff, negative]]
9         [[food, neutral], [miscellaneous, positive]]
Name: output_normalized, dtype: object


In [79]:
none_values = df_normalized[df_normalized['input'].isnull()]

# Print out the rows with None values in the 'input' column
print(none_values)

Empty DataFrame
Columns: [input, output_normalized]
Index: []


In [80]:
# Sample data
data = [
    (['It might be the best sit down food I\'ve had in the area, so if you are going to the upright citizen brigade, or the garden, it could be just the place for you.'], ('food', 'positive'))
]

# Processing each row in the data
for row in data:
    text_list, label_pair = row
    text = text_list[0]  # Extract the string from the list
    aspect, sentiment = label_pair
    print(f"Aspect: {aspect}, Sentiment: {sentiment}")

    start_idx = text.find(aspect)
    if start_idx != -1:
        end_idx = start_idx + len(aspect)
        print(f"Aspect '{aspect}' found in text at position {start_idx} to {end_idx}.")
    else:
        print(f"Aspect '{aspect}' not found in text.")


Aspect: food, Sentiment: positive
Aspect 'food' found in text at position 30 to 34.


In [81]:
df_normalized['input'].head(1)

0    [It might be the best sit down food I've had i...
Name: input, dtype: object

In [82]:
df_normalized.head()

,input,output_normalized
0,[It might be the best sit down food I've had i...,"[[food, positive], [place, neutral]]"
1,[Hostess was extremely accommodating when we a...,"[[staff, positive], [miscellaneous, neutral]]"
2,[We were a couple of minutes late for our rese...,"[[miscellaneous, neutral], [staff, negative]]"
3,"[Though the service might be a little slow, th...","[[service, negative], [staff, positive]]"
4,[Although we arrived at the restaurant 10 min ...,"[[staff, negative], [miscellaneous, neutral]]"


In [83]:
import ast

def clean_aspect_sentiment(row):
    # Parse the input text and aspect-sentiment pairs
    try:
        if isinstance(row['input'], str):
            text = ast.literal_eval(row['input'])[0] if isinstance(ast.literal_eval(row['input']), list) else row['input']
        elif isinstance(row['input'], list) and len(row['input']) > 0:
            text = row['input'][0]
        else:
            raise ValueError("Invalid input format")

        aspect_sentiments = row['output_normalized']
        if isinstance(aspect_sentiments, str):
            aspect_sentiments = ast.literal_eval(aspect_sentiments)
        elif not isinstance(aspect_sentiments, list):
            raise ValueError("Invalid output format")

        # Remove pairs with null aspects or sentiments
        cleaned_aspect_sentiments = [pair for pair in aspect_sentiments if None not in pair]

        return text, cleaned_aspect_sentiments
    except Exception as e:
        print(f"Error processing row {row.name}: {e}")
        return None, None

# Apply the cleaning function to each row
cleaned_data = df_normalized.apply(clean_aspect_sentiment, axis=1)

# Remove rows where the cleaning function returned None
cleaned_data = cleaned_data.dropna()

# Create separate lists for texts and aspect-sentiment pairs
texts = cleaned_data.apply(lambda x: x[0]).tolist()
aspect_sentiments = cleaned_data.apply(lambda x: x[1]).tolist()

# Verify the cleaned data
for text, aspects in zip(texts, aspect_sentiments):
    print(f"Text: {text}")
    print(f"Aspects: {aspects}")


Streaming output truncated to the last 5000 lines.
Text: I wouldn't play a first-person shooter with this, mind, but if you wanted to run MS Office, email, chat, download a video, listen to music from a certain fruit-named music store, and were looking for a highly portable yet powerful netbook to do that all in, I'd highly recommend checking this out.
Aspects: [['MS Office', 'neutral']]
Text: and looks very sexyy:D  really the mac book pro is the best laptop specially for students in college if you are not caring about price.
Aspects: [['price', 'negative']]
Text: The only thing I wish this had was the option to turn off the touchpad with a button like my big 16" laptop does.
Aspects: [['touchpad', 'negative']]
Text: The only thing i can say is that the touch pad doesnt work like it should all the time.
Aspects: [['touch pad', 'negative']]
Text: My husband uses it mostly for games, email and music.
Aspects: [['games', 'neutral']]
Text:   Until I bought the Dell, I thought you just loo

In [84]:
print(aspect_sentiments)

[[['food', 'positive'], ['place', 'neutral']], [['staff', 'positive'], ['miscellaneous', 'neutral']], [['miscellaneous', 'neutral'], ['staff', 'negative']], [['service', 'negative'], ['staff', 'positive']], [['staff', 'negative'], ['miscellaneous', 'neutral']], [['miscellaneous', 'negative'], ['food', 'neutral']], [['food', 'neutral'], ['price', 'positive'], ['miscellaneous', 'neutral']], [['miscellaneous', 'negative'], ['food', 'neutral']], [['miscellaneous', 'neutral'], ['staff', 'negative']], [['food', 'neutral'], ['miscellaneous', 'positive']], [['miscellaneous', 'positive'], ['staff', 'positive'], ['food', 'neutral']], [['food', 'negative'], ['miscellaneous', 'neutral']], [['food', 'neutral'], ['miscellaneous', 'positive']], [['service', 'negative'], ['staff', 'positive']], [['staff', 'negative'], ['miscellaneous', 'neutral']], [['food', 'neutral'], ['staff', 'negative']], [['menu', 'positive'], ['place', 'neutral']], [['food', 'neutral'], ['place', 'positive']], [['staff', 'posit

In [85]:
lowercased_strings = [s.lower() for s in texts]

In [86]:
lowercased_aspect_sentiment = [
    [[aspect[0].lower(), aspect[1]] for aspect in group] for group in aspect_sentiments
]

In [87]:
from collections import Counter
import pandas as pd

# Flatten the list and count occurrences
flat_list = [tuple(item) for sublist in lowercased_aspect_sentiment for item in sublist]
counts = Counter(flat_list)

# Convert the counts to a DataFrame with an added 'Conflict' category
data_vs = {
    'Aspect': [],
    'Positive': [],
    'Neutral': [],
    'Negative': [],
    'Conflict': []
}

# Initialize counters for each aspect and sentiment
for aspect, sentiment in counts:
    if aspect not in data_vs ['Aspect']:
        data_vs ['Aspect'].append(aspect)
        data_vs ['Positive'].append(0)
        data_vs ['Neutral'].append(0)
        data_vs ['Negative'].append(0)
        data_vs ['Conflict'].append(0)
    index = data_vs ['Aspect'].index(aspect)
    if sentiment == 'positive':
        data_vs ['Positive'][index] = counts[(aspect, sentiment)]
    elif sentiment == 'neutral':
        data_vs ['Neutral'][index] = counts[(aspect, sentiment)]
    elif sentiment == 'negative':
        data_vs ['Negative'][index] = counts[(aspect, sentiment)]
    elif sentiment == 'conflict':
        data_vs ['Conflict'][index] = counts[(aspect, sentiment)]

# Create the DataFrame
df = pd.DataFrame(data_vs)

In [88]:
print(df)

                                 Aspect  Positive  Neutral  Negative  Conflict
0                                  food      1644     1959       542        16
1                                 place       271      537       272         5
2                                 staff       517      161      1232         2
3                         miscellaneous       286      692       241         0
4                               service       582      206       744        13
...                                 ...       ...      ...       ...       ...
4476  lamb glazed with balsamic vinegar         1        0         0         0
4477                       sichuan chef         0        1         0         0
4478       meat patties in steamed buns         0        0         1         0
4479                          sake list         1        0         0         0
4480                        purple haze         0        0         0         1

[4481 rows x 5 columns]


In [89]:
Data = pd.DataFrame({
    'input': lowercased_strings,
    'output_normalized': lowercased_aspect_sentiment,
})

In [90]:
import nltk

In [91]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [92]:
import nltk
from nltk.tokenize import word_tokenize

# Make sure the Punkt tokenizer is available
nltk.download('punkt')

def bio_tagging(text, aspects):
    tokens = word_tokenize(text)
    bio_tags = ['O'] * len(tokens)  # Initialize all tags to 'Outside'

    # Debugging output
    print("Tokens:", tokens)

    for aspect_info in aspects:
        aspect = aspect_info[0]  # The aspect term
        aspect_tokens = word_tokenize(aspect)

        # Debugging output
        print("Aspect Tokens:", aspect_tokens)

        # Start tagging
        for i in range(len(tokens)):
            if tokens[i:i+len(aspect_tokens)] == aspect_tokens:
                bio_tags[i] = 'B-ASP'  # Begin tag
                for j in range(1, len(aspect_tokens)):
                    if i+j < len(bio_tags):
                        bio_tags[i+j] = 'I-ASP'  # Inside tag

        # Debugging output to check what is tagged
        print("Tags after processing aspect '{}':".format(aspect), bio_tags)

    return list(zip(tokens, bio_tags))

# Example usage
text = "It might be the best sit down food I've had in the area, so if you are going to the upright citizen brigade, or the garden, it could be just the place for you."
aspects = [['food', 'positive'], ['place', 'neutral']]
bio_tagged = bio_tagging(text, aspects)
print(bio_tagged)


Tokens: ['It', 'might', 'be', 'the', 'best', 'sit', 'down', 'food', 'I', "'ve", 'had', 'in', 'the', 'area', ',', 'so', 'if', 'you', 'are', 'going', 'to', 'the', 'upright', 'citizen', 'brigade', ',', 'or', 'the', 'garden', ',', 'it', 'could', 'be', 'just', 'the', 'place', 'for', 'you', '.']
Aspect Tokens: ['food']
Tags after processing aspect 'food': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ASP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Aspect Tokens: ['place']
Tags after processing aspect 'place': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ASP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ASP', 'O', 'O', 'O']
[('It', 'O'), ('might', 'O'), ('be', 'O'), ('the', 'O'), ('best', 'O'), ('sit', 'O'), ('down', 'O'), ('food', 'B-ASP'), ('I', 'O'), ("'ve", 'O'), ('had', 'O'), ('in', 'O'), ('the', 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [93]:
def bio_tagging_sep(text, aspects):
    tokens = word_tokenize(text)
    bio_tags = ['O'] * len(tokens)  # Initialize all tags to 'Outside'

    # Find and tag each aspect in the text
    for aspect_info in aspects:
        aspect = aspect_info[0]
        aspect_tokens = word_tokenize(aspect)

        # Search for the exact sequence of aspect tokens in the sentence tokens
        start_index = 0  # Initialize the starting index for searching
        while start_index < len(tokens):
            try:
                # Attempt to find the starting index of the aspect tokens sequence
                start_index = tokens.index(aspect_tokens[0], start_index)
                # Check if the following tokens match the aspect tokens
                if tokens[start_index:start_index + len(aspect_tokens)] == aspect_tokens:
                    bio_tags[start_index] = 'B-ASP'
                    for i in range(1, len(aspect_tokens)):
                        bio_tags[start_index + i] = 'I-ASP'
                # Move the start index forward to continue searching
                start_index += len(aspect_tokens)
            except ValueError:
                # If the aspect token isn't found, break the loop
                break

    return tokens, bio_tags

In [94]:
result_series = Data.apply(lambda row: bio_tagging_sep(row['input'], row['output_normalized']), axis=1)

# Convert the Series of tuples into a DataFrame
result_df = result_series.apply(pd.Series)
result_df.columns = ['tokens', 'bio_tags']  # Naming the columns

# Concatenate the new DataFrame with the original DataFrame
Data = pd.concat([Data, result_df], axis=1)

print(Data.head())

                                               input  \
0  it might be the best sit down food i've had in...   
1  hostess was extremely accommodating when we ar...   
2  we were a couple of minutes late for our reser...   
3  though the service might be a little slow, the...   
4  although we arrived at the restaurant 10 min l...   

                               output_normalized  \
0           [[food, positive], [place, neutral]]   
1  [[staff, positive], [miscellaneous, neutral]]   
2  [[miscellaneous, neutral], [staff, negative]]   
3       [[service, negative], [staff, positive]]   
4  [[staff, negative], [miscellaneous, neutral]]   

                                              tokens  \
0  [it, might, be, the, best, sit, down, food, i,...   
1  [hostess, was, extremely, accommodating, when,...   
2  [we, were, a, couple, of, minutes, late, for, ...   
3  [though, the, service, might, be, a, little, s...   
4  [although, we, arrived, at, the, restaurant, 1...   

             

In [95]:
Data['bio_tagged']

KeyError: 'bio_tagged'

In [ ]:
Data['output_normalized'].iloc[1]

In [ ]:
Data['input'].iloc[1]

In [ ]:
Data.head()

In [ ]:
Data.shape

In [ ]:
filtered_data = Data[Data['extracted_aspects'].map(len) > 0]

print(filtered_data)

In [ ]:
filtered_data.head()

In [ ]:
filtered_data = filtered_data.loc[:,~filtered_data.columns.duplicated()]

In [ ]:
filtered_data.head()
filtered_data.to_csv('DataIOBTagged', index=False)

In [ ]:
import numpy as np
X_tokens = filtered_data['tokens'].tolist()  # Convert 'tokens' column to a list
Y_bio_tags = filtered_data['bio_tags'].tolist()

In [ ]:
print(Y_bio_tags[:5])

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer




tag_tokenizer = Tokenizer()
tag_tokenizer.fit_on_texts(Y_bio_tags)  # This maps BIO tags to integers
Y_encoded = tag_tokenizer.texts_to_sequences(Y_bio_tags)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 85  # This should match the maxlen used for X
Y_padded = pad_sequences(Y_encoded, maxlen=max_len, padding='post')  # Padding 'post' to align with X padding

In [ ]:
from tensorflow.keras.utils import to_categorical

num_tags = len(tag_tokenizer.word_index) + 1  # Number of different tags
Y_one_hot = to_categorical(Y_padded, num_classes=num_tags)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Assuming X_tokens is a list of lists of words (e.g., [['hello', 'world'], ['test', 'data']])
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_tokens)  # This creates a mapping of words to indices

# Convert texts to sequences of integers
X_encoded = tokenizer.texts_to_sequences(X_tokens)


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming you decide on a maximum sequence length (you can determine this based on your data distribution)
max_length = max(len(x) for x in X_encoded)  # You could set this manually if you want shorter sequences

# Pad sequences so that they all have the same length
X_padded = pad_sequences(X_encoded, maxlen=85, padding='post')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
cd drive

In [ ]:
cd MyDrive/

In [ ]:
import numpy as np

def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Path to your GloVe file in Google Drive
glove_file = 'glove.6B.300d.txt'
embeddings_index = load_glove_embeddings(glove_file)
print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
embedding_dim = 300  # This should match the dimension of the GloVe vectors you are using
vocab_size = len(word_tokenizer.word_index) + 1  # +1 for padding (0 index)

# Initialize the embedding matrix with zeros
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Populate the embedding matrix with GloVe vectors
for word, i in word_tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed, Bidirectional

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix],
                    input_length=85, trainable=False))  # Set trainable to False to not fine-tune embeddings
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(TimeDistributed(Dense(num_tags, activation='softmax')))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_padded, Y_one_hot, test_size=0.2, random_state=42)  # Using 20% of the data for testing


In [ ]:
model.fit(X_padded, Y_one_hot, batch_size=32, epochs=10)

In [ ]:
print(X_padded.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed, Bidirectional

model2 = Sequential()
model2.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix],
                    input_length=85, trainable=False))
model2.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(TimeDistributed(Dense(3, activation='softmax')))

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.summary()


In [ ]:
from tensorflow.keras.utils import to_categorical

Y__sentiment = to_categorical(Y_padded_sentiment, num_classes=3)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_tokens)

X_padded_sentiment = tokenizer.texts_to_sequences(X_tokens)


In [ ]:
model2.fit(X_padded_sentiment, Y__sentiment, batch_size=32, epochs=10)